# Programe sua GPU com OpenMP

Autores: 
_Hermes Senger_ e
_Jaime Freire de Souza_

## Configuração do ambiente

Precisaremos de um compilador capaz de gerar código executável para GPUs. 


In [1]:
#%%shell
#ln -sfnv /usr/local/cuda-10.1/ /usr/local/cuda
#pip install -q matplotlib numpy
#wget https://openmp-course.s3.amazonaws.com/llvm.tar.gz
#tar -xzvf llvm.tar.gz >/dev/null 2>&1
#echo "  ------------  Terminou a instalação! Pode continuar  ------------------"

In [2]:
import os

os.environ['LLVM_PATH'] = '/content/llvm'
os.environ['PATH'] = os.environ['LLVM_PATH'] + '/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = os.environ['LLVM_PATH'] + '/lib:' + os.environ['LD_LIBRARY_PATH']
os.environ['TSAN_OPTIONS'] = 'ignore_noninstrumented_modules=1'


Vamos testar nosso ambiente de programação. 

In [3]:
%%writefile test.c

#include <omp.h>
#include <stdio.h>

int main() {
  int numdevices = omp_get_num_devices();
  printf("number of devices= %d\n", numdevices);
}

Writing test.c


In [4]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda test.c -o teste

!./teste

 [-Wlinker-warnings]
1 warning generated.
number of devices= 4


Vamos verificar o modelo de GPU alocada. 

In [5]:
!nvidia-smi

Tue Apr 12 09:49:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:18:00.0 Off |                  N/A |
| 25%   36C    P0    39W / 250W |      0MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 26%   

## Cálculo de Pi - versão serial

O programa a seguir calcula o valos re Pi pelo método de integração numérica. Esta primeira versão trabalha de forma serial, somente na CPU. Nas próximas versões, nós faremos melhorias nesse programa para acelerar o seu desempenho. 


In [6]:
%%writefile ex1-pi_serial.c
// The OpenMP Common Core - pg. 58
// https://github.com/tgmattso/OmpCommonCore/blob/master/Book/C/Fig_4.5_poprogseq.c
#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();

   for (i = 0; i < num_steps; i++){
      x = (i + 0.5) * step;
      sum += 4.0 / (1.0 + x * x);
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}	

Writing ex1-pi_serial.c


In [7]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda ex1-pi_serial.c -o ex1-pi_serial

!./ex1-pi_serial

 [-Wlinker-warnings]
1 warning generated.
pi = 3.141593, 100000000 steps, 1.459288 secs
 

## Uma versão paralela para CPU - Pi-V1.0.c

O próximo exemplo mostra uma forma de paralelizar o algoritmo para execução em CPU, utilizando os seguintes recursos do OpenMP:

* Regiões paralelas
* Loops paralelos
* Redução

In [8]:
%%writefile Pi-V1.0.c

#include <stdio.h>
#include <omp.h>
//static long num_steps = 100000000;
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp parallel 
   {
     double x;        /* cada thread terá sua variavel x local */
     #pragma omp for reduction(+:sum)
       for (i = 0; i < num_steps; i++){
          x = (i + 0.5) * step;
          sum += 4.0 / (1.0 + x * x);
       }
   }
   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%20.15lf, \%ld steps, \%lf secs\n ", pi, num_steps, run_time);
}	

Overwriting Pi-V1.0.c


In [9]:
#%%shell

#!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda Pi-V1.0.c -o Pi-V1.0
!clang -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0
!clang -O2 -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0
!clang -O3 -fopenmp Pi-V1.0.c -o Pi-V1.0
!./Pi-V1.0

pi =    3.141592653589808, 100000000 steps, 0.116168 secs
 pi =    3.141592653589808, 100000000 steps, 0.044143 secs
 pi =    3.141592653589808, 100000000 steps, 0.056025 secs
 

# Usando a GPU - Pi melhorado V2.0

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construção __target__ :

  `#pragma omp target`    
  `#pragma omp parallel for`   
  `     for (i=0;i<N;i++) ...`

  Modifique o programa abaixo, introduzindo a melhoria sugerida: 

In [10]:
%%writefile pi-par-V2.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
 
   // Remover esta linha para versão dos alunos
   #pragma omp target map(sum)
   {
     // Remover esta linha para versão dos alunos
     #pragma omp parallel for reduction(+: sum) private(x)
     for (i = 0; i < num_steps; i++){
       x = (i + 0.5) * step;
       sum += 4.0 / (1.0 + x * x);
      }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}	

Overwriting pi-par-V2.c


In [11]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda  pi-par-V2.c -o pi-par-V2

!./pi-par-V2

 [-Wlinker-warnings]
1 warning generated.
pi = 3.141593, 100000000 steps, 2.169342 secs
 


__Pergunta:__ O que aconteceu? Compare o tempo de execução com o da CPU.

__Resposta__:  Nesta versão, somente __um time de threads__ foi criado, para executar blocos de iterações do loop. Os threads desse time executarão de forma paralela, mas ocupando apenas um __compute unit__ apenas, o resultado não será muito bom.  

# Usando a GPU - Pi melhorado V3.0

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construções __target__, __teams__ e __distribute__ :

  `#pragma omp target`    
  `#pragma omp teams`    
  `#pragma omp distribute`    
  `     for (i=0;i<N;i++) ...`


  Modifique o programa abaixo, introduzindo a melhoria sugerida. 

  __Obs:__
  1. Para este exercício, não utilize a construção  `#pragma omp parallel for`. 
  2. Modifique o número de iterações para apenas 1000 (ou seja, reduzir por um fator de 10E-5).


In [12]:
%%writefile pi-par-V3.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp target map(sum)
   #pragma omp teams reduction(+: sum)
    {
     double x;       
     #pragma omp distribute
     for (i = 0; i < num_steps; i++){
       x = (i + 0.5) * step;
       sum += 4.0 / (1.0 + x * x);
     }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}	

Overwriting pi-par-V3.c


In [13]:
#%%shell

#!clang -O3 -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda  pi-par-V3.c -o pi-par-V3
!clang -fopenmp -fopenmp-targets=nvptx64  pi-par-V3.c -o pi-par-V3

!./pi-par-V3

pi = 3.141593, 100000000 steps, 2.334968 secs
 

Compare o tempo de execução com o anterior e explique por que foi mais lento que o anterior. 

__Resposta__: Neste exemplo, foram criados 128 teams (depende do compilador/hardware) com apenas 1 thread (chamado de thread inicial) para cada team. 

# Usando a GPU - Pi melhorado V4.0

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construções __teams distribute__ e __parallel for__:

  `#pragma omp target`    
  `#pragma omp teams distribute`  
  `#pragma omp parallel for` 
    `for (i=0;i<N;i++) ...`
     

Modifique o programa abaixo, introduzindo a melhoria sugerida. 

  __Obs:__
  1. Para este exercício, não utilize a construção  `#pragma omp parallel for`. 
  2. Modifique o número de iterações para apenas 1000 (ou seja, reduzir por um fator de 10E-5).


In [14]:
%%writefile pi-par-V4.c

#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp target map(sum)
   {
     #pragma omp teams distribute parallel for reduction(+:sum) private(x)  
     for (i = 0; i < num_steps; i++){
       x = (i + 0.5) * step;
       sum += 4.0 / (1.0 + x * x);
     }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}	

Overwriting pi-par-V4.c


In [15]:
#%%shell

#!clang -O3 -fopenmp -fopenmp-targets=nvptx64-nvidia-cuda  pi-par-V3.c -o pi-par-V3
!clang  -fopenmp -fopenmp-targets=nvptx64  pi-par-V4.c -o pi-par-V4

!./pi-par-V4

pi = 3.141593, 100000000 steps, 1.677457 secs
 

In [16]:
!clang --version

clang version 14.0.0 (https://github.com/llvm/llvm-project.git c47135949779904542ee1e6d5486609ebf0da1b5)
Target: x86_64-unknown-linux-gnu
Thread model: posix
InstalledDir: /opt/llvm/bin


# Usando a GPU - Pi melhorado V5.0

A seguir, vamos utilizar a GPU para tentar acelerar nosso programa, utilizando:
* Construções __teams distribute__ , __parallel for simd__:

  `#pragma omp target`    
  `#pragma omp teams distribute`  
  `for (i=0;i<N;i++) ...`
  `#pragma omp parallel for` 
    `for (i=0;i<block_sizeN;i++) ...`
     

Modifique o programa abaixo, introduzindo a melhoria sugerida. 

...


In [17]:
%%writefile pi-par-V5.c
#define MIN(x, y) (((x) < (y)) ? (x) : (y))
#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;

   start_time = omp_get_wtime();
   #pragma omp target map(sum)
   #pragma omp teams reduction(+:sum) 
   {
     int block_size = num_steps/omp_get_num_teams();
//     #pragma omp distribute dist_sched(static, 1)
     #pragma omp distribute        
     for (int ii = 0; ii < num_steps; ii += block_size){
         #pragma omp parallel for simd reduction(+: sum)
//         #pragma omp parallel for reduction(+: sum)         
         for (int i = ii; i < MIN(ii+block_size, num_steps); i++) {
             x = (i + 0.5) * step;
             sum += 4.0 / (1.0 + x * x);
         }
     }
   }

   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}	

Overwriting pi-par-V5.c


In [18]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64  pi-par-V5.c -o pi-par-V5

!./pi-par-V5

pi = 3.141593, 100000000 steps, 0.179204 secs
 

# Usando a GPU - Pi melhorado V6.0

Juntando tudo isso 

In [21]:
%%writefile pi-par-V6.c
#define MIN(x, y) (((x) < (y)) ? (x) : (y))
#include <stdio.h>
#include <omp.h>
static long num_steps = 100000000;
double step;
int main ()
{
   int i;
   double x, pi, sum = 0.0;
   double start_time, run_time;

   step = 1.0 / (double) num_steps;
   start_time = omp_get_wtime();
   #pragma omp target map(sum)
//   #pragma omp teams distribute parallel for simd reduction(+: sum) private(x)
   #pragma omp teams distribute parallel for reduction(+: sum) private(x)    
   for (i=0; i< num_steps; i++){
             x = (i + 0.5) * step;
             sum += 4.0 / (1.0 + x * x);
   }
   pi = step * sum;
   run_time = omp_get_wtime() - start_time;
   printf("pi = \%lf, \%ld steps, \%lf secs\n ",
                pi, num_steps, run_time);
}	

Overwriting pi-par-V6.c


In [22]:
#%%shell

!clang -fopenmp -fopenmp-targets=nvptx64  pi-par-V6.c -o pi-par-V6

!./pi-par-V6

pi = 3.141593, 100000000 steps, 1.674195 secs
 